In [1]:
import pandas as pd
import numpy as np
import math
import sympy as sym

In [2]:
x = sym.Symbol('x')
y = sym.Symbol('y')
alpha = sym.Symbol('alpha')

In [3]:
F = x**2 + 4*y**2 + 2*x - 4*y - 2

### Вариант 1

### Метод наискорейшего спуска

### $$t_{n+1} = t_n - \alpha_n*f'(t_n) \\ (1)$$

In [4]:
#задаем начальные параметры
x_n = -0.9
y_n = 0.6
eps = 1;
n = 0
while eps > 0.01:  #пока не выполнится правило остановки
    n+=1
    print("step:", n)
    
    #подсчитываем значения производных в точках и подставляем в формулу (1)
    f_x = x_n - alpha * F.diff(x).subs([(x, x_n),(y, y_n)])
    f_y= y_n - alpha * F.diff(y).subs([(x, x_n),(y, y_n)])
    
    #подставляем полученые значения новой точки в исходную функцию
    F_alpha = F.subs([(x, f_x),(y, f_y)])
    
    #находим производную по альфа и решаем уравнение, чтобы найти минимум
    alpha_n = sym.solve(sym.simplify(F_alpha).diff(alpha))
    print("alpha = ",alpha_n[0])
    
    #подставляем найденное значение альфа в уравнение (1)
    x_n = f_x.subs(alpha, alpha_n[0])
    y_n = f_y.subs(alpha, alpha_n[0])
    print("X:",x_n)
    print("Y:",y_n)
    
    #проверяем правило остановки
    eps = max(abs(F.diff(x).subs([(x, x_n),(y, y_n)])), abs(F.diff(y).subs([(x, x_n),(y, y_n)])))
    print("Eps = ",eps)
    print()


step: 1
alpha =  0.130769230769231
X: -0.926153846153846
Y: 0.495384615384615
Eps =  0.147692307692308

step: 2
alpha =  0.424999999999999
X: -0.988923076923077
Y: 0.511076923076923
Eps =  0.0886153846153848

step: 3
alpha =  0.130769230769231
X: -0.991820118343195
Y: 0.499488757396450
Eps =  0.0163597633136097

step: 4
alpha =  0.424999999999998
X: -0.998773017751479
Y: 0.501226982248521
Eps =  0.00981585798816553



Значения схожи с вичислениями в отчете. Отличаются из-за погрешностей в вычислениях.

### Метод сопряженных градиентов

### $$t_{n+1} = t_n + \alpha_n*h(t_n) \\ (2)$$

### $$h_n = -f'(t_{n}) + \beta_{n-1}*h_{n-1} \\ (3)$$
### $$ h_0 = -f'(t_0) \\ (4)$$

### $$ \beta_n = \frac{(A*h_n,f'(t_{n+1})}{(A*h_n,h_n)} \\ (5)$$

In [5]:
#задаем начальные параметры
x_n = -0.9
y_n = 0.6
n = 0
A = sym.Matrix([[1,0],[0,4]])
h = sym.Matrix([0,0])

#достаточно не более N итераций, где N - количество переменных
for i in range(2):
    n+=1
    print("step: ", n)
    
    # если первая итерация то используем формулу (4), иначе (3)
    if n == 1:
        
        # находим h_0
        h[0] = - F.diff(x).subs([(x, x_n),(y, y_n)])
        h[1] = - F.diff(y).subs([(x, x_n),(y, y_n)])
        
        # подставляем в уравнение (2)
        f_x = x_n + alpha * h[0]
        f_y = y_n + alpha * h[1]
    else:
        #считаем Beta (5)
        Beta = (sym.Matrix([F.diff(x).subs([(x, x_n),(y, y_n)]), F.diff(y).subs([(x, x_n),(y, y_n)])]).transpose()*A*h)[0]/ (h.transpose()*A*h)[0]
        
        # находим h_n
        h[0] = - F.diff(x).subs([(x, x_n),(y, y_n)])+Beta * h[0]
        h[1] = - F.diff(y).subs([(x, x_n),(y, y_n)])+Beta * h[1]
        
        # подставляем в уравнение (2)
        f_x = x_n + alpha * h[0]
        f_y = y_n + alpha * h[1]
    
    #подставляем полученые значения новой точки в исходную функцию
    F_alpha = F.subs([(x, f_x),(y, f_y)])
    
    #находим производную по альфа и решаем уравнение, чтобы найти минимум
    alpha_n = sym.solve(sym.simplify(F_alpha).diff(alpha))
    print("alpha = ",alpha_n[0])
    
    #подставляем найденное значение альфа в уравнение (1)
    x_n = f_x.subs(alpha, alpha_n[0])
    y_n = f_y.subs(alpha, alpha_n[0])
    print("X: ",x_n)
    print("Y: ",y_n)
    
    #находим отклонение от искомого решения: (-1,0.5)
    print("error = ",((x_n+1.)**2+(y_n-0.5)**2)**0.5)
    print()

step:  1
alpha =  0.130769230769231
X:  -0.926153846153846
Y:  0.495384615384615
error =  0.0739902440394525

step:  2
alpha =  0.477941176470588
X:  -1.00000000000000
Y:  0.500000000000000
error =  0



Значения схожи с вичислениями в отчете. Отличаются из-за погрешностей в вычислениях.